# US Treasury Yield Curve — Nelson–Siegel Fit
This notebook loads Treasury bond data, computes bond yields to maturity, fits a Nelson–Siegel yield curve, and visualizes the fitted curve alongside observed yields.

## 1. Read Data
Data are read from `US_Treasury_Data.xlsx` (sheet `Bonds`). Dates are parsed and mid prices are computed from bid/ask.

In [38]:
import numpy as np 
import pandas as pd 
from scipy.optimize import newton, minimize 
import matplotlib.pyplot as plt 

# ----------------------------------------------------------- 
# 1. READ DATA 
# ----------------------------------------------------------- 

treasury_bonds = pd.read_excel("US_Treasury_Data.xlsx", sheet_name="Bonds") 
treasury_bonds['Issue Date'] = pd.to_datetime(treasury_bonds['Issue Date']) 
treasury_bonds['Maturity'] = pd.to_datetime(treasury_bonds['Maturity']) 

# Use mid price (Bid/Ask midpoint) 
treasury_bonds["Price"] = (treasury_bonds["Bid Price"] + treasury_bonds["Ask Price"]) / 2 


## 2. Build YTM Solver
Yield-to-maturity is solved per bond using Newton's method on the pricing equation.

In [ ]:
 
 # ----------------------------------------------------------- 
 # 2. BUILD YTM SOLVER FOR EACH BOND 
 # ----------------------------------------------------------- 
 
def compute_ytm(price, coupon_rate, maturity_years, freq=2, FV=100): 
    """ 
    Computes yield-to-maturity using Newton's method. 
    
    price : clean price (no AI) 
    coupon_rate : annual coupon % (e.g. 3.5 for 3.5%) 
    maturity_years : time to maturity in years 
    freq : number of coupon payments per year (Treasury = 2) 
    """ 
    c = (coupon_rate / 100) * FV / freq 
    n = int(maturity_years * freq) 

    def price_from_yield(y): 
        pv_coupons = sum(c / (1 + y/freq)**t for t in range(1, n+1)) 
        pv_face = FV / (1 + y/freq)**n 
        return pv_coupons + pv_face 

    # Root-finding objective 
    f = lambda y: price_from_yield(y) - price 

    # Initial guess 
    try: 
        return newton(f, x0=0.03, maxiter=100) 
    except: 
        return np.nan 


## 3. Time to Maturity
Compute time to maturity in years and filter out expired bonds.

In [ ]:
 
 # ----------------------------------------------------------- 
 # 3. CLEAN UP + COMPUTE TIME TO MATURITY 
 # ----------------------------------------------------------- 
 
today = pd.Timestamp.today() 

treasury_bonds["Tau"] = (treasury_bonds["Maturity"] - today).dt.days / 365 
treasury_bonds = treasury_bonds[treasury_bonds["Tau"] > 0]  # remove expired 


## 4. Calculate YTM
Apply the YTM solver across bonds and drop missing yields.

In [ ]:
 
 # ----------------------------------------------------------- 
 # 4. CALCULATE YTM 
 # ----------------------------------------------------------- 
 
ytms = [] 
for _, row in treasury_bonds.iterrows(): 
    ytm = compute_ytm( 
        price=row["Price"], 
        coupon_rate=row["Cpn"], 
        maturity_years=row["Tau"], 
        freq=2 
    ) 
    ytms.append(ytm) 

treasury_bonds["YTM"] = ytms 

# Drop NA YTMs 
treasury_bonds = treasury_bonds.dropna(subset=["YTM"]) 


## 5. Nelson–Siegel Model
Define the Nelson–Siegel functional form and the sum-of-squared-errors objective.

In [ ]:
 
 # ----------------------------------------------------------- 
 # 5. NELSON–SIEGEL MODEL 
 # ----------------------------------------------------------- 
 
def nelson_siegel(tau, beta0, beta1, beta2, lamb): 
    """ 
    NS yield curve function y(t). 
    """ 
    term1 = (1 - np.exp(-lamb * tau)) / (lamb * tau) 
    term2 = term1 - np.exp(-lamb * tau) 
    return beta0 + beta1 * term1 + beta2 * term2 


# Objective = SSE 
def ns_objective(params, tau, ytm_obs): 
    beta0, beta1, beta2, lamb = params 
    ytm_pred = nelson_siegel(tau, beta0, beta1, beta2, lamb) 
    return np.sum((ytm_obs - ytm_pred)**2) 


## 6. Fit NS Parameters
Estimate parameters by minimizing SSE between observed YTMs and the model.

In [ ]:
 
 # ----------------------------------------------------------- 
 # 6. FIT NS PARAMETERS 
 # ----------------------------------------------------------- 
 
tau_data = treasury_bonds["Tau"].values 
ytm_data = treasury_bonds["YTM"].values 

initial_guess = [0.05, -0.02, 0.01, 0.5] 

result = minimize(ns_objective, initial_guess, args=(tau_data, ytm_data)) 

beta0, beta1, beta2, lamb = result.x 
SSE = result.fun 

print("
=== NELSON–SIEGEL PARAMETERS ===") 
print(f"Level (β0):   {beta0}") 
print(f"Slope (β1):   {beta1}") 
print(f"Curvature (β2): {beta2}") 
print(f"Decay (λ):     {lamb}") 
print(f"SSE:           {SSE}") 
print("================================
") 


## 7. Plot Fitted Yield Curve
Visualize the fitted curve against the observed bond YTMs.

In [ ]:
 
 # ----------------------------------------------------------- 
 # 7. PLOT FITTED YIELD CURVE 
 # ----------------------------------------------------------- 
 
tau_grid = np.linspace(0.05, 30, 500) 
fitted_curve = nelson_siegel(tau_grid, beta0, beta1, beta2, lamb) 

plt.figure(figsize=(12, 6)) 
plt.plot(tau_grid, fitted_curve, color = "red", label="Nelson–Siegel Fitted Curve", linewidth=2) 
plt.scatter(tau_data, ytm_data, color='blue', label="Actual Bond YTMs") 
plt.title("US Treasury Yield Curve - Nelson–Siegel Fit") 
plt.xlabel("Time to Maturity (Years)") 
plt.ylabel("Yield") 
plt.grid(True) 
plt.legend() 
plt.show() 
